In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
config = {
    "api_type": os.getenv("AZURE_OPENAI_API_TYPE"),
    "api_base": os.getenv("AZURE_OPENAI_API_BASE"),
    "api_version": os.getenv("AZURE_OPENAI_API_VERSION"),
    "api_key": os.getenv("AZURE_OPENAI_API_KEY"),
    "deployment_name": os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    "model_name": os.getenv("AZURE_OPENAI_MODEL"),
}

In [2]:
import openai
from langchain_openai import AzureOpenAIEmbeddings

AZURE_OPENAI_API_KEY = config["api_key"]
AZURE_OPENAI_ENDPOINT = config["api_base"]
AZURE_OPENAI_DEPLOYMENT_NAME = "text-embedding-ada-002"

openai.api_type = config["api_type"]
openai.api_key = config["api_key"]
openai.api_base = config["api_base"]
openai.api_version = config["api_version"]

embeddings = AzureOpenAIEmbeddings(
    openai_api_key=AZURE_OPENAI_API_KEY,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    openai_api_version=config["api_version"],
    deployment=AZURE_OPENAI_DEPLOYMENT_NAME,
    openai_api_type=config["api_type"],
)

In [3]:
from PyPDF2 import PdfReader

pdfreader = PdfReader('./pdf/99speedmart.pdf')

In [ ]:
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

raw_text = "\n".join([line for line in raw_text.splitlines() if line.strip()])

raw_text

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

documents = [Document(page_content=raw_text)]
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1200,
    chunk_overlap=200
)
texts = text_splitter.split_documents(documents)

texts

In [ ]:
# -----------------------------
# Run Embeddings
# -----------------------------
embedding_data = []

# Generate embeddings for each text chunk
for idx, doc in enumerate(texts):
    # Access the page_content of the Document
    text = doc.page_content.strip()  # Assuming texts are Document objects
    if not text:  # Skip empty chunks
        continue
    
    try:
        # Generate the embedding for the current text chunk
        embedding = embeddings.embed_query(text)  # Ensure this method exists
        embedding_data.append({
            'chunk': idx + 1,
            'text': text,
            'embedding': embedding
        })
        print(f"Generated embedding for Chunk {idx + 1}")
    except Exception as e:
        print(f"Error generating embedding for Chunk {idx + 1}: {e}")

# Optionally, print the resulting embedding data
for data in embedding_data:
    print(f"Chunk {data['chunk']}:")
    print(f"Text: {data['text']}")
    print(f"Embedding: {data['embedding']}")
    print("-" * 40)

In [ ]:
for item in embedding_data:
    print(f"Chunk {item['chunk']} Embedding:\n{item['embedding']}\n{'-'*50}")

In [ ]:
import numpy as np
import faiss
import pickle

embeddings_list = [item['embedding'] for item in embedding_data]
embeddings_matrix = np.array(embeddings_list).astype('float32')  # Faiss requires float32

# Extract metadata
metadata = [item['text'] for item in embedding_data]

# Determine the dimensionality of the embeddings
dimension = embeddings_matrix.shape[1]

# Initialize the Faiss index
index = faiss.IndexFlatL2(dimension)

# Verify if GPU is available and use it
if faiss.get_num_gpus() > 0:
    print("Using GPU for Faiss index")
    res = faiss.StandardGpuResources()
    index = faiss.index_cpu_to_gpu(res, 0, index)
else:
    print("Using CPU for Faiss index")


# Add embeddings to the Faiss index
index.add(embeddings_matrix)
print(f"Number of vectors in the index: {index.ntotal}")

# -----------------------------
# Persisting Faiss Index and Metadata
# -----------------------------

# Save the index to a file
faiss.write_index(index, "faiss_index.index")
print("Faiss index saved to faiss_index.index")

# Save metadata
with open("metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)
print("Metadata saved to metadata.pkl")

In [9]:
import faiss, pickle
import numpy as np

# -----------------------------
# Loading Faiss for Saved Index & Metadata
# -----------------------------


index = faiss.read_index("faiss_index.index")
if faiss.get_num_gpus() > 0:
    res = faiss.StandardGpuResources()
    index = faiss.index_cpu_to_gpu(res, 0, index)
with open("metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

In [21]:
# -----------------------------
# Ask a Question
# -----------------------------
query = "What is the share price in 2019?"

In [ ]:
# -----------------------------
# Performing a Similarity Search
# -----------------------------

# Generate embedding for the query
try:
    query_embedding = embeddings.embed_query(query)
    query_embedding_np = np.array([query_embedding]).astype('float32')
except Exception as e:
    print(f"Error generating embedding for query: {e}")

# Number of nearest neighbors
k = 3

# Perform the search
distances, indices = index.search(query_embedding_np, k)

# Display the results
for i in range(k):
    idx = indices[0][i]
    distance = distances[0][i]
    print(f"Rank {i + 1}:")
    print(f"Text: {metadata[idx]}")
    print(f"Distance: {distance}\n")

In [ ]:
# -----------------------------
# Answer with LLM
# -----------------------------
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_openai import AzureChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

documents = [Document(page_content=text, metadata={}) for text in metadata]

vector_store = FAISS.from_documents(documents=documents, embedding=embeddings)

llm = AzureChatOpenAI(
    deployment_name=config["deployment_name"],
    model_name=config["model_name"],
    azure_endpoint=config["api_base"],
    openai_api_key=config["api_key"],
    openai_api_version=config["api_version"],
    openai_api_type=config["api_type"],
    temperature=0.1,
    max_tokens=150,
)

system_prompt = (
'''
You are a financial advisor. Answer the question based solely on the context below:

<context>

{context}

</context>
'''
)
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
chain = create_retrieval_chain(vector_store.as_retriever(), question_answer_chain)

result = chain.invoke({"input": query})

print(f"Question: {query}\n")
print(f"Answer: {result['answer']}")